SETUP

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
input_path = "D:/temp_files/datasets/customer_segmentation/Online Retail.xlsx"
output_path =  "D:/temp_files/datasets/customer_segmentation/freshman.xlsx"
data = pd.read_excel(input_path)
temp = data.copy(deep=True)

Data Exploration

<https://www.zhihu.com/question/505784526>

idx = list[np.where(data['CustomerID'].str.contains) 条件获取索引

查询df中数量小于0的行，提取index并转换成list

idx = data_temp.query('Quantity <= 0').index.tolist()

In [20]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Quantity     406829 non-null  int64         
 1   InvoiceDate  406829 non-null  datetime64[ns]
 2   UnitPrice    406829 non-null  float64       
 3   CustomerID   406829 non-null  float64       
 4   Country      406829 non-null  object        
 5   total_price  406829 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 21.7+ MB


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


<https://www.kaggle.com/code/mohamedattia98/full-data-science-project/notebook>
* InvoiceNo invoice number
* StockCode	unique code for each transaction
* Description description of individual item
* Quantity number of items brought
* InvoiceDate data of purchase
* UnitPrice Price of individual unit
* CustomerID unique ID for each customer
* Country country name

缺失值以及无关行列的处理
https://archive.ics.uci.edu/ml/datasets/online+retail


In [2]:
# 去除ID列的nan值，在原数据集里表示为空白
temp.dropna(subset='CustomerID', inplace=True)


In [3]:
# 去除无用列，暂时尝试不按照商品种类划分
features_droped = ['Description', 'StockCode', 'InvoiceNo']
temp.drop(labels=features_droped, axis=1, inplace=True)


获取光顾频率

In [18]:
# 按ID分组，获取日期的最大最小值
grouped_date = temp.groupby(['CustomerID']).InvoiceDate.agg([min, max])
# 获取光临的总时间间隔
grouped_date['interval'] = pd.to_datetime(grouped_date['max']) - pd.to_datetime(grouped_date['min']) 
# 当天来多次，时间天数间隔为0，故+1
grouped_date['interval'] = grouped_date['interval'].apply(lambda row: row.days+1)
# 获取总计光顾次数
grouped_date = pd.concat([grouped_date, temp.groupby(['CustomerID']).InvoiceDate.agg([len])], axis=1)
# 在此标准下，计算光顾频率
grouped_date['frequency'] = grouped_date['len'] / grouped_date['interval']
grouped_date.drop(labels=['min','max','len','interval'], axis=1, inplace=True)

获取总额度，注意到，有一些Quantity（商品数）的值为负数，可以推测不是错误记录，但具体物理含义需要查阅资料

In [16]:
# 原数据列中增加total行，表示当前交易行为的总交易额
temp['total_price'] = temp.Quantity * temp.UnitPrice
# 按ID分组后，算出该分组内的总额度
grouped_total_price = temp.groupby(['CustomerID']).total_price.sum()
grouped_total_quantity = temp.groupby(['CustomerID']).Quantity.sum()
# 获取总额度/数量
grouped_total = pd.concat(axis=1, objs=[grouped_total_price, grouped_total_quantity])
# 交易数量为0，但是交易额为正数，此处假设为交易人白赚了一笔
grouped_total.Quantity.replace(to_replace=0, value=1, inplace=True)
grouped_total['price_d_quantity'] = grouped_total['total_price'] / grouped_total['Quantity']


现在尝试数据进行特征缩放、编码

In [19]:
grouped = pd.concat(objs=[grouped_date, grouped_total], axis=1)
del grouped_total_price, grouped_total_quantity, grouped_date

In [27]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_grouped = scaler.fit_transform(grouped)
scaled_grouped = pd.DataFrame(index=grouped.index,data=scaled_grouped, columns=grouped.columns)

代码输出到文件，从而保存预处理的结果，这是一个新手预处理

In [ ]:
scaled_grouped.to_excel(output_path, sheet_name='Sheet1', index=True)